# COMP9318 Lab4

## Instructions
1. This note book contains instructions for **COMP9318-Lab4**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures via corresponding functions.

* You can submit your implementation for **Lab4** via following link: https://kg.cse.unsw.edu.au/submit/ .

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* You are allowed to add other functions and/or import modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **23:59:59 on 16th April, 2019**. We will **not** accept any late submissions.

# Question-1: Text Classification using Multinomial Naive Bayes

You are required to implement a multinomial naive bayes classifier to predict spam SMS.

The training data is a set of SMS categoried into `spam` and `ham`.

In [3]:
import pandas as pd

raw_data = pd.read_csv('./asset/data.txt', sep='\t')
raw_data.head()

,category,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In order to implement a unigram model, first we tokenize the text. We use the count corresponding to each token (word) in the SMS as its feature (i.e., bag of words). We store the features and catrgorical information for each SMS in a `dictionary`.

In [4]:
def tokenize(sms):
    return sms.split(' ')

def get_freq_of_tokens(sms):
    tokens = {}
    for token in tokenize(sms):
        if token not in tokens:
            tokens[token] = 1
        else:
            tokens[token] += 1
    return tokens

training_data = []
for index in range(len(raw_data)):
    training_data.append((get_freq_of_tokens(raw_data.iloc[index].text), raw_data.iloc[index].category))

For this lab, you need to **implement** a multinomial naive bayes classifier (i.e., `multinomial_nb()` in the file: `submission.py`) with add-1 smoothing. The input arguments of `multinomial_nb()` are:
* `training_data`: pre-processed data stored as a `dictionary`
* `sms`: test-sms (i.e., a list of tokens) that you need to categorize as `spam` and/or `ham`

The return value of `multinomial_nb()` should be the **ratio** of the probability of sms is spam and the probability of sms is ham. A return value larger than 1 implies the `sms` is spam and vice versa.

For example, a sample output is shown in the cell given below:

In [3]:
## How we test your implementation...
import submission_ans as submission

sms = 'I am not spam'
print(submission.multinomial_nb(training_data, tokenize(sms)))

0.2342767295597484




# Test Environment

For testing, we have pre-installed the requisite modules and/or libraries in the testing environment. You are only allowed to use following libraries:
* python: 3.6.5
* pandas: 0.19.2

NOTE: You are required to implement the classifier by yourself. You are not allowed to import **sklearn** and/or any other library in Lab4.

In [5]:
def merge_two(x,y):
    if(x == {}):
        return y
    if(y == {}):
        return x
    l = list(x.keys())+list(y.keys())
    z={}
    for i in l:
        if(i in y and i in x):
            z[i] = x[i]+y[i]
        elif(i in x):
            z[i] = x[i]
        else:
            z[i] = y[i]
    return z
        
            #0.2342767295597484

In [6]:
import pandas as pd
import math


def multinomial_nb(training_data,sms):
    #calculate class probability
    Pham=0
    Pspam=0
    hamList=[]
    spamList=[]
    for i in training_data:
        if(list(i)[1]=='ham'):
            Pham+=1
            hamList.append(list(i)[0])
        else:
            spamList.append(list(i)[0])
            Pspam+=1
    Pham = Pham/(len(training_data))
    Pspam = Pspam/(len(training_data))
    
    ###Probability of each word per class
    result_ham = {}
    result_spam = {}
    #when ham show all words
    for d_1 in hamList:
        result_ham = merge_two(result_ham,d_1)
    #when spam show all words
    for d_2 in spamList:
        result_spam = merge_two(result_spam,d_2)
    #number of  words ham class
    s_ham = sum(result_ham.values())
    #number of  words spam class
    s_spam = sum(result_spam.values())
    #occuency of each word
    result_total = result_ham.copy()
    result_total.update(result_spam)

    total_words = len(result_total)
    #calculate the Probability of each word in spam/ham class
    #ham:
    Prob_word_ham={}
    for word in result_total:
        if(word in result_ham):
            Prob_word_ham[word] = (result_ham[word]+1)/(s_ham+total_words)
        else:
            Prob_word_ham[word] = 1/(s_ham+total_words)
    #spam:
    Prob_word_spam={}
    for word in result_total:
        if(word in result_spam):
            Prob_word_spam[word] = (result_spam[word]+1)/(s_spam+total_words)
        else:
            Prob_word_spam[word] = 1/(s_spam+total_words)     
    #prediction
    #spam probability
    product = 1
    for word in sms:
        if(word in Prob_word_spam):
            product = product*Prob_word_spam[word]
    spam_pro = Pspam*product
    product = 1
    #ham probability
    for word in sms:
        if(word in Prob_word_ham):
            product = product*Prob_word_ham[word]
    ham_pro = Pham*product    
    print('spam prob is:',spam_pro)
    print('ham prob is:',ham_pro)

    return spam_pro/ham_pro

In [7]:
sms = 'I am not spam'
multinomial_nb(training_data, tokenize(sms))

spam prob is: 0.0020964360587002098
ham prob is: 0.008948545861297539


0.23427672955974846